## import libraries

### pip install

In [1]:
#pip install
!pip install faiss-gpu --quiet
!pip install boto3 --quiet
!pip install langchain-aws --quiet
!pip install langchain-community boto3 --quiet
!pip install boto3 requests requests-aws4auth --quiet
!pip install pymongo --quiet
!pip install google-search-results sagemaker --quiet
!pip install boto3 nltk --quiet
!pip install wikipedia --quiet
!pip install tavily-python --quiet
!pip install --upgrade langchain sympy numexpr --quiet
!pip install langchain-experimental --quiet
!pip install -U langgraph langchain_community langchain_anthropic langchain_experimental --quiet

### libraries

In [2]:
import boto3
import os
from langchain import hub

#chat models
from langchain_aws import ChatBedrock
from langchain_aws import ChatBedrockConverse

#LLMs
from langchain_aws import BedrockLLM
from langchain_community.llms import AmazonAPIGateway
from langchain_aws import SagemakerEndpoint

#prompts
from langchain.prompts import PromptTemplate

#embedding models
from langchain_aws import BedrockEmbeddings
# from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import ContentHandlerBase

#document loaders
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
from langchain_community.document_loaders import AmazonTextractPDFLoader

#vector stores
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.vectorstores import DocumentDBVectorSearch

from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

#retrievers
from langchain_aws import AmazonKendraRetriever
from langchain_aws import AmazonKnowledgeBasesRetriever

#memory
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory

#graphs
from langchain_community.graphs import NeptuneGraph
from langchain_community.graphs import NeptuneAnalyticsGraph
from langchain_community.chains.graph_qa.neptune_cypher import NeptuneOpenCypherQAChain

from langchain_community.graphs import NeptuneRdfGraph
from langchain_community.chains.graph_qa.neptune_sparql import NeptuneSparqlQAChain

#callbacks
from langchain_community.callbacks.bedrock_anthropic_callback import BedrockAnthropicTokenUsageCallbackHandler
from langchain_community.callbacks import SageMakerCallbackHandler

#chains
# from langchain_experimental.comprehend_moderation import AmazonComprehendModerationChain

#tavily
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults

#tools
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.tools import BaseTool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, CSVLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

#agent
from langchain.agents import create_tool_calling_agent, AgentType
from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI

USER_AGENT environment variable not set, consider setting it to identify your requests.


## setup agent

In [3]:
model_parameter = {"temperature": 0.9, "top_p": .5, "max_tokens_to_sample": 200}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3.client('bedrock'),
    model_kwargs=model_parameter, 
    beta_use_converse_api=True
)

In [4]:
embeddings = BedrockEmbeddings(
    client=boto3.client('bedrock-runtime', region_name="us-east-1"),
    model_id = "amazon.titan-embed-text-v2:0"
)

### initiate tools

In [5]:
os.environ["TAVILY_API_KEY"] = "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"
search = TavilySearchResults()

In [6]:
# Define the path to the directory containing CSV files
data_files_directory = "data_files"

# Use DirectoryLoader to load all CSV files
loader = DirectoryLoader(
    data_files_directory,
    glob="**/*.csv",  # Match all CSV files in the directory (including subdirectories)
    loader_cls=CSVLoader  # Use CSVLoader for parsing the files
)

In [7]:
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "Retriever_tool",
    "First search internal data files to answer any queries. If there is no useful information then use other tools.",
)

In [8]:
# prompt = PromptTemplate(
#     input_variables=["input", "agent_scratchpad"],  # Include both input and agent_scratchpad
#     template="""
# Use the retriever tool to answer any queries related to internal data files. 
# Only use external tools (like Wikipedia) if no relevant information is found internally.
# When asked about a specific item output the answers to the following queries: 

# Should this item be ordered? 
# Use recent stock statistics to see if the stocks for this month are low compared to how much was used last month.
# Use the statistics from last year to predict how much of each item will be needed for the rest of this month and next month.
# Use what month it is at the time the call is being made and whether any important holidays eg christmas or summer will be coming up and use this information in your decision.

# What brand?
# Use the customers reviews to decide this. 

# If this item should be ordered, how much of this item should be ordered?
# Use the repl tool to calculate these numbers.

# In your final output:
# 1. Explain your reasoning for whether an item should be ordered and how much.
# Don't explain your reasoning for
# Explain the maths you used.
# 2. Draft an email to an imaginary supplier of this brand to order the number of the item you think should be ordered, signing off as Reply Auto Replenishment.

# Query: {input}

# {agent_scratchpad}
# """
# )


In [9]:
prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],  # Include both input and agent_scratchpad
    template="""
    
If the input is to do with retail or an item to buy, do the following:

Use the retriever tool to answer any queries related to internal data files. 
Only use external tools (like Wikipedia) if no relevant information is found internally.
When asked about a specific item output the answers to the following queries: 

Should this item be ordered? 
Find out what date it is currently using date time tool. Convert the month and year into strings and use these for the rest of the answer.
Use recent stock statistics to see if the stocks for this month are low compared to how much was used last month.
Use the statistics from last year to predict how much of each item will be needed for the rest of this month and next month.

Use wikipedia tool to find out whether any important holidays eg christmas or summer will be coming up and mention this search.
Use wikipedia search information and the date information in your decisions.
a) how much should be ordered for this month and next month
b) how much should be ordered in the next few months

What brand?
Use the customers reviews to decide this. 

If this item should be ordered, how much of this item should be ordered?
Use the repl tool to calculate these numbers.

In your final output:
1. Explain your reasoning for whether an item should be ordered and how much.
Don't explain your reasoning for
Explain the maths you used.
2. Draft an email to an imaginary supplier of this brand to order the number of the item you think should be ordered, signing off as Reply Auto Replenishment.

Else:
Answer the question appropriately using multiple tools to verify your answer

Query: {input}

{agent_scratchpad}
"""
)


In [10]:
# math_prompt = PromptTemplate(
#     input_variables=["input"],
#     template="Calculate the result of the following expression and return it as a valid Python expression: {input}"
# )
# math_tool = load_tools(["llm-math"], llm=bedrock_llm,prompt=math_prompt)



In [11]:
#datetime tool
from datetime import datetime
datetime_tool = Tool(
    name="Datetime",
    func=lambda x: datetime.now().isoformat(),
    description="Returns the current datetime",
)


In [12]:
#loading in tools

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Use this to execute python commands and calculations.",
    func=python_repl.run,
)
wikipedia_tool = load_tools(["wikipedia"], llm=bedrock_llm)

tools = [datetime_tool,retriever_tool,search,repl_tool,wikipedia_tool[0]]

## customer agent

In [13]:
customer_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],  # Include both input and agent_scratchpad
    template="""
    whatever item is inputted, return three clients's Ids that it would be best to advertise this item to.
    use retriever tool to look through list of clients in supermarket.csv file.
    Return their client ids, found in the first column on the csv file.
    
    using tavilly search / wikipedia / repl tool do the following:
    identify what the item is, what is made of, and benefits from this product if applicable.
    come up with the best way to advertise it to them - using factors such as their name and their spending category.
    give clients that have shopped at the company for longer a greater discount and mention this fact.
Query: {input}

{agent_scratchpad}
"""
)


## running agents

In [14]:
def run_agent(prompt,agent_input,raw_list,verbose_setting):
    agent = create_tool_calling_agent(bedrock_llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose_setting,handle_parsing_errors=True)
    response = agent_executor.invoke({"input": agent_input})
    raw_output = response['output'][0]['text']
    # raw_list.append(raw_output)
    print(raw_output)

In [15]:
raw_list = []
#run_agent(prompt,'suncream',raw_list,False)
run_agent(prompt,'christmas pudding',raw_list,False)
#run_agent(customer_prompt,'cereal',raw_list,False)
#run_agent(customer_prompt,'bread',raw_list,False)

# for i in range(len(raw_list)):
#     print('-'*20)
#     print(raw_list[i])
#     print('\n')

Python REPL can execute arbitrary code. Use with caution.


Here is my analysis and recommendation for ordering Christmas pudding:

Based on the internal data, Christmas pudding is a seasonal item that is only sold in December. Last December, a total of 20 units were sold across the three brands (Brand A: 12 units, Brand B: 8 units, Brand C: 0 units). 

Looking at this month (December 2024), we only have 2 units left in stock, while last month (November) we used 5 units. Since the stock left this month is less than 20% of what was used last month, our stocks are considered low for this seasonal item.

To predict our needs for this month and next, I looked at sales data from last year. In December 2023, we sold 20 units total. In January 2024, we sold 0 units. So I will predict we need 20 units for this month (December 2024) and 0 units for next month (January 2025).

Given we only have 2 units left, I recommend ordering an additional 18 units for this month to meet the predicted demand of 20 units. For next month, no additional ordering is need

In [16]:
# customer_agent = create_tool_calling_agent(bedrock_llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)
# response = agent_executor.invoke({"input": 'sun cream'})
# raw_output = response['output'][0]['text']

In [17]:
# agent = create_tool_calling_agent(bedrock_llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)
# response = agent_executor.invoke({"input": 'sun cream'})
# raw_output = response['output'][0]['text']

# print(raw_output)

## multi agent system

### creating agent supervisor

In [21]:
from typing import Literal
from typing_extensions import TypedDict

from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState
from langgraph.types import Command


members = ["researcher", "communication"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]

system_prompt = f"""You are a supervisor tasked with managing a conversation between the
    following workers: {members}. Given the following user request,
    respond with the worker to act next. Each worker will perform a
    task and respond with their results and status. 
    
    Do not let the workers attempt the task more than 10 times, stop them before the recursive limit of 25 is reached.
    
    When a good email has been written by communication node, stop the workers,
    respond with FINISH, and output the final email."""

class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[options]


def supervisor_node(state: MessagesState) -> Command[Literal[members, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = bedrock_llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END
    
    print('hello',response)
    # return Command(goto=goto)
    return response

### construct graph

In [22]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

# Define agents
# research_agent = create_react_agent(
#     bedrock_llm, tools=tools, state_modifier="You are a researcher. You do the maths and predictions."
# )

research_string = """
    
If the input is to do with retail or an item to buy, do the following:

Use the retriever tool to answer any queries related to internal data files. 
Only use external tools (like Wikipedia) if no relevant information is found internally.
When asked about a specific item output the answers to the following queries: 

Should this item be ordered? 
Find out what date it is currently using date time tool. Convert the month and year into strings and use these for the rest of the answer.
Use recent stock statistics to see if the stocks for this month are low compared to how much was used last month.
Use the statistics from last year to predict how much of each item will be needed for the rest of this month and next month.
Use what month it is at the time the call is being made and whether any important holidays eg christmas or summer will be coming up and use this information in your decision.
a) how much should be ordered for this month and next month
b) how much should be ordered in the next few months


What brand?
Use the customers reviews to decide this. 

If this item should be ordered, how much of this item should be ordered?
Use the repl tool to calculate these numbers.

Query: {input}

{agent_scratchpad}
"""

communication_string = """
    
Using information from the research node, 
Draft an email to an imaginary supplier of this brand to order the number of the item you think should be ordered, 
signing off as Reply Auto Replenishment.

Query: {input}

{agent_scratchpad}
"""

research_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],  # Include both input and agent_scratchpad
    template=research_string
)



communication_prompt= PromptTemplate(
    input_variables=["input", "agent_scratchpad"],  # Include both input and agent_scratchpad
    template=communication_string
)


research_agent = create_react_agent(
    bedrock_llm, tools=tools, state_modifier=research_string
)

def research_node(state: MessagesState) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )

# Communication node definition
communication_agent = create_react_agent(bedrock_llm, tools=[search,wikipedia_tool[0]],state_modifier=communication_string)

def communication_node(state: MessagesState) -> Command[Literal["supervisor"]]:
    result = communication_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="communication")
            ]
        },
        goto="supervisor",
    )

# Supervisor node definition
# def supervisor_node(state: MessagesState) -> Command[Literal["researcher", "communication", END]]:
#     # Dummy implementation of supervisor logic
#     return Command(update={}, goto="researcher")  # Example logic

# Build StateGraph
builder = StateGraph(MessagesState)

# Add nodes explicitly with corresponding functions
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("communication", communication_node)

# Add edges explicitly
builder.add_edge(START, "supervisor")
builder.add_edge("supervisor", "researcher")
builder.add_edge("researcher", "supervisor")
builder.add_edge("supervisor", "communication")
builder.add_edge("communication", "supervisor")
builder.add_edge("supervisor", END)

# Compile graph
try:
    graph = builder.compile()
except TypeError as e:
    print("Error during graph compilation:", e)
    # Debugging spec.ends if needed
    for node_name, spec in builder.nodes.items():
        print(f"Node: {node_name}, Ends: {spec.ends}")


Error during graph compilation: unhashable type: 'list'
Node: supervisor, Ends: (['researcher', 'communication'], '__end__')
Node: researcher, Ends: ('supervisor',)
Node: communication, Ends: ('supervisor',)


In [20]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

NameError: name 'graph' is not defined

### invoke team

In [ ]:
for s in graph.stream(
    {"messages": [("user", "Bread")]}, subgraphs=True
):
    print(s)
    print("----")

In [ ]:
from typing import Literal
from typing_extensions import TypedDict
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent

# Define team members and options
members = ["researcher", "communication"]
options = members + ["FINISH"]

# Supervisor system prompt
system_prompt = f"""You are a supervisor managing a conversation between the workers: {members}. 
You decide which worker should act next based on the current task progress. 
Stop all work and respond with FINISH when a good email has been written by the communication node. 
Output the final email when the task is complete."""

# Supervisor routing logic
class Router(TypedDict):
    next: Literal[options]

def supervisor_node(state: MessagesState) -> Command[Literal[members + [END]]]:
    messages = state["messages"]
    # Check if email is complete
    for msg in messages[::-1]:
        if "email completed" in msg.get("content", "").lower():
            final_email = msg["content"]
            return Command(update={"final_email": final_email}, goto=END)
    # Decide next step based on the last message sender
    last_sender = messages[-1].get("name", "") if messages else ""
    if last_sender == "researcher":
        return Command(update={}, goto="communication")
    elif last_sender == "communication":
        return Command(update={}, goto="researcher")
    return Command(update={}, goto="researcher")  # Default to researcher

# Researcher prompt
research_string = """
If the input is to do with retail or an item to buy, do the following:

Use the retriever tool to answer any queries related to internal data files. 
Only use external tools (like Wikipedia) if no relevant information is found internally.
When asked about a specific item output the answers to the following queries: 

Should this item be ordered? 
Find out what date it is currently using date time tool. Convert the month and year into strings and use these for the rest of the answer.
Use recent stock statistics to see if the stocks for this month are low compared to how much was used last month.
Use the statistics from last year to predict how much of each item will be needed for the rest of this month and next month.
Use what month it is at the time the call is being made and whether any important holidays eg christmas or summer will be coming up and use this information in your decision.
a) how much should be ordered for this month and next month
b) how much should be ordered in the next few months

What brand?
Use the customers reviews to decide this. 

If this item should be ordered, how much of this item should be ordered?
Use the repl tool to calculate these numbers.

Query: {input}

{agent_scratchpad}
"""

research_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"], template=research_string
)

# Researcher node
def research_node(state: MessagesState) -> Command[Literal["supervisor"]]:
    research_agent = create_react_agent(bedrock_llm, tools=tools, state_modifier=research_string)
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )

# Communication prompt
communication_string = """
Using the information from the researcher, draft an email to a supplier for ordering items.
Sign off as Reply Auto Replenishment.

Query: {input}

{agent_scratchpad}
"""

communication_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"], template=communication_string
)

# Communication node
def communication_node(state: MessagesState) -> Command[Literal["supervisor"]]:
    communication_agent = create_react_agent(bedrock_llm, tools=[search, wikipedia_tool[0]], state_modifier=communication_string)
    result = communication_agent.invoke(state)
    content = result["messages"][-1].content
    if "good email" in content.lower():
        content += "\nStatus: Email completed."
    return Command(
        update={
            "messages": [
                HumanMessage(content=content, name="communication")
            ]
        },
        goto="supervisor",
    )

# Build the StateGraph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("communication", communication_node)

# Add edges
builder.add_edge(START, "supervisor")
builder.add_edge("supervisor", "researcher")
builder.add_edge("researcher", "supervisor")
builder.add_edge("supervisor", "communication")
builder.add_edge("communication", "supervisor")
builder.add_edge("supervisor", END)

# Compile the graph
try:
    graph = builder.compile()
except TypeError as e:
    print("Error during graph compilation:", e)

# Stream execution
for s in graph.stream(
    {"messages": [("user", "Bread")]}, subgraphs=True
):
    print(s)
    print("----")


## making own math tool

In [ ]:
from langchain.chains import LLMMathChain, LLMChain
from langchain_community.utilities import WikipediaAPIWrapper
from dotenv import load_dotenv
load_dotenv()

wikipedia = WikipediaAPIWrapper()
wikipedia_tool = Tool(name="Wikipedia",
 func=wikipedia.run,
 description="A useful tool for searching the Internet

word_problem_template = """You are a reasoning agent tasked with solving 
the user's logic-based questions. Logically arrive at the solution, and be 
factual. In your answers, clearly detail the steps involved and give the 
final answer. Provide the response in bullet points. 
Question  {question} Answer"""

math_assistant_prompt = PromptTemplate(input_variables=["question"],
                                       template=word_problem_template
                                       )

word_problem_chain = LLMChain(llm=bedrock_llm,
                              prompt=math_assistant_prompt)

word_problem_tool = Tool.from_function(name="Reasoning Tool",
                                       func=word_problem_chain.run,
                                       description="Useful for when you need to answer logic-based/reasoning questions.",
 )


problem_chain = LLMMathChain.from_llm(llm=bedrock_llm)

math_tool = Tool.from_function(name="Calculator",
    func=problem_chain.run,
    description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.")


agent = initialize_agent(
    tools=[wikipedia_tool[0], math_tool, word_problem_tool],
    llm=bedrock_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True
 )

print(agent.invoke({"input": """I have 3 apples and 4 oranges. 
     I give half of my oranges away and buy two dozen new ones, along
     with three packs of strawberries. Each pack of strawberry has 30 strawberries.
     How  many total pieces of fruit do I have at the end?"""}))

## csv writing agent

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent

csv_agent = create_csv_agent(
    llm,
    "draft_emails_agent.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

## experimenting

In [ ]:
agent = initialize_agent(tools, llm=bedrock_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
#result = agent.invoke({'input':"What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?"})
inputs = {
    "page_content": "Content from the document",
    "input": "What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?"
}
result = agent.invoke(inputs)

In [ ]:
math_query = "What is the square root of 256 multiplied by 10?"
mq2="What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?"

In [ ]:
response = agent_executor.invoke({"input": '3+3'})

## call agent

In [ ]:
math = agent_executor.invoke({'input':'what is 13 +20'})

In [ ]:
response = agent_executor.invoke({"input": "Toothpaste"})

In [ ]:
from textwrap import dedent
# Extract the raw text output
raw_output = response['output'][0]['text']

print(raw_output)

In [ ]:
# Format the output (example formatting logic)
def format_output(raw_text):
    # Parse raw text to extract relevant sections (if structured)
    lines = raw_text.split("\n")
    formatted_text = dedent(f"""
    **Order Summary**

    1. Should this item be ordered?
       {lines[3]}

    2. **Recommended Brand:**
       {lines[4]}
       - Review Highlights:
           - {lines[5]}
           - {lines[6]}

    **Draft Email to Supplier**
    {lines[-2]}

    **Next Steps:**
    1. Confirm availability with the supplier.
    2. Place the order.
    3. Follow up on delivery.
    """)
    return formatted_text

# Format the response
formatted_response = format_output(raw_output)

# Display the formatted response
print(formatted_response)